In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import sqlite3
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import uuid

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_google_vertexai import VertexAIEmbeddings

from typing import TypedDict
from dataclasses import dataclass
from langgraph.runtime import get_runtime
from langgraph.checkpoint.memory import InMemorySaver
from langchain_community.utilities import SQLDatabase
from langchain_core.tools import tool
from langchain.agents import create_agent



from dotenv import load_dotenv
load_dotenv()

/Users/sawale/Documents/learning/money_rag/.venv/lib/python3.12/site-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


True

In [3]:
import os 
import json
import asyncio
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize search
search_tool = DuckDuckGoSearchRun()
MERCHANT_CACHE_FILE = "merchant_cache.json"

# Helper to load/save cache
def _load_cache():
    if os.path.exists(MERCHANT_CACHE_FILE):
        try:
            with open(MERCHANT_CACHE_FILE, 'r') as f:
                return json.load(f)
        except:
            return {}
    return {}

def _save_cache(cache):
    with open(MERCHANT_CACHE_FILE, 'w') as f:
        json.dump(cache, f)

async def get_merchant_category_async(description, cache, sem):
    """Async search with semaphore and shared cache"""
    if description in cache:
        return cache[description]
    
    async with sem:
        try:
            # Random small sleep to jitter requests slightly
            await asyncio.sleep(0.05) 
            print(f"   🔍 Web searching for: {description}...")
            # Run the search asynchronously
            result = await search_tool.ainvoke(f"What type of business / store is '{description}'? Whats is location")
            return result
        except Exception as e:
            print(f"   ⚠️ Search failed for {description}: {e}")
            return "Unknown"

async def ingest_csv(file_path, llm, db_path="money_rag.db"):
    print(f"📂 Processing {file_path}...")
    df = pd.read_csv(file_path)
    headers = df.columns.tolist()
    sample_data = df.head(10).to_json() 

    # 1. LLM Mapping (Sync is fine here)
    prompt = ChatPromptTemplate.from_template("""
    Act as a financial data parser. Analyze this CSV data:
    Filename: {filename}
    Headers: {headers}
    Sample Data: {sample}

    TASK:
    1. Map the CSV columns to standard fields: date, description, amount, and category.
    2. Determine the 'sign_convention' for spending.
       
       RULES:
       - If the filename suggests 'Discover' credit card, spending are usually POSITIVE.
       - If the filename suggests 'Chase' credit card, spending are usually NEGATIVE.
                                              
       - Analyze the 'sign_convention' for spending (outflows):
          - Look at the sample data for known merchants or spending patterns.
          - If spending (like a restaurant or store) is NEGATIVE (e.g., -25.00), the convention is 'spending_is_negative'.
          - If spending is POSITIVE (e.g., 25.00), the convention is 'spending_is_positive'.

    OUTPUT FORMAT (JSON ONLY):
    {{
      "date_col": "column_name",
      "desc_col": "column_name",
      "amount_col": "column_name",
      "category_col": "column_name or null",
      "sign_convention": "spending_is_negative" | "spending_is_positive"
    }}
    """)
    
    chain = prompt | llm | JsonOutputParser()
    mapping = chain.invoke({"headers": headers, "sample": sample_data, "filename": file_path})

    # 2. Standardization
    standard_df = pd.DataFrame()
    standard_df['id'] = [str(uuid.uuid4()) for _ in range(len(df))]
    standard_df['transaction_date'] = pd.to_datetime(df[mapping['date_col']])
    standard_df['description'] = df[mapping['desc_col']]
    
    # Normalization Logic
    raw_amounts = pd.to_numeric(df[mapping['amount_col']])
    if mapping['sign_convention'] == "spending_is_negative":
        standard_df['amount'] = raw_amounts * -1
    else:
        standard_df['amount'] = raw_amounts
    
    standard_df['category'] = df[mapping['category_col']] if mapping.get('category_col') else 'Uncategorized'
    standard_df['source_file'] = file_path.split("/")[-1]

    # 3. --- Async Enrichment Step ---
    print("   ✨ Enriching descriptions (Async)...")
    unique_descriptions = standard_df['description'].unique()
    
    # Load cache once
    cache = _load_cache()
    
    # Create a semaphore to limit concurrent web searches (e.g. 5)
    sem = asyncio.Semaphore(5)
    
    # Create tasks for all descriptions
    # Note: We process ALL descriptions now effectively
    tasks = []
    for desc in unique_descriptions:
        tasks.append(get_merchant_category_async(desc, cache, sem))
        
    # Run in parallel
    results = await asyncio.gather(*tasks)
    
    # Update cache object with new results
    for desc, res in zip(unique_descriptions, results):
        cache[desc] = res
        
    # Save cache back to disk
    _save_cache(cache)
        
    # Map back
    desc_map = dict(zip(unique_descriptions, results))
    standard_df['enriched_info'] = standard_df['description'].map(desc_map).fillna("")

    # 4. Save to DB
    conn = sqlite3.connect(db_path)
    standard_df.to_sql("transactions", conn, if_exists="append", index=False)
    conn.close()
    
    print(f"✅ Ingested {len(standard_df)} rows from {file_path.split('/')[-1]}. Logic: {mapping['sign_convention']}")

In [4]:
path1 = "/Users/sawale/Documents/learning/money_rag/demo_data/Discover-AllAvailable-20260110.csv"
path2 = "/Users/sawale/Documents/learning/money_rag/demo_data/Chase5282_Activity20240110_20260110_20260111.CSV"

# Initialize the Gemini model via Vertex AI
vertex_llm = init_chat_model(
    "gemini-2.5-flash", 
    model_provider="google_vertexai",
    project='gen-lang-client-0311515393',
    location='us-central1',
)

# Run async functions in Jupyter
# We run them sequentially here to avoid file lock issues with SQLite if both try to write at once
# (Though SQLite handles concurrency, keeping ingestion strictly ordered is safer for the demo)
await ingest_csv(path1, vertex_llm)
await ingest_csv(path2, vertex_llm)

📂 Processing /Users/sawale/Documents/learning/money_rag/demo_data/Discover-AllAvailable-20260110.csv...
   ✨ Enriching descriptions (Async)...
✅ Ingested 124 rows from Discover-AllAvailable-20260110.csv. Logic: spending_is_positive
📂 Processing /Users/sawale/Documents/learning/money_rag/demo_data/Chase5282_Activity20240110_20260110_20260111.CSV...
   ✨ Enriching descriptions (Async)...
✅ Ingested 126 rows from Chase5282_Activity20240110_20260110_20260111.CSV. Logic: spending_is_negative


In [5]:

import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("money_rag.db")

# Query all transactions
df_view = pd.read_sql_query("SELECT * FROM transactions", conn)

# Close connection
conn.close()

# Display the data
df_view

,id,transaction_date,description,amount,category,source_file,enriched_info
0,5712be16-98a8-41d7-94de-15ecff7793da,2024-10-17 00:00:00,BACK MARKET BROOKLYN NY,231.19,Merchandise,Discover-AllAvailable-20260110.csv,"Online Retailer in New York, NY . See BBB rati..."
1,cea08bf8-440c-4d05-b06c-5c0d2e3b2997,2024-10-18 00:00:00,TEMU.COM 8884958368 DE,16.51,Merchandise,Discover-AllAvailable-20260110.csv,Temu ' s business model has allowed it to beco...
2,b021b290-36fc-4667-993c-6f620da3a54a,2024-10-18 00:00:00,WALMART STORE 00332 HUNTSVILLE AL,146.73,Merchandise,Discover-AllAvailable-20260110.csv,Walmart Inc. is an American multinational reta...
3,d260bc87-ca63-4238-ad9e-3482273c49fb,2024-10-18 00:00:00,$100 STATEMENT CREDIT W 1ST PU,-100.00,Awards and Rebate Credits,Discover-AllAvailable-20260110.csv,All U.S. Bank credit cards offer contactless c...
4,2c06fe8e-8ef2-49f1-bed6-dd282effec85,2024-11-02 00:00:00,PY *KUNG-FU TEA AL HUNTSVILLE AL,8.09,Restaurants,Discover-AllAvailable-20260110.csv,"Jan 22, 2021 · Best part to me--besides the ro..."
...,...,...,...,...,...,...,...
245,6a6167f7-dc54-480f-8a15-0561c7b877da,2025-06-18 00:00:00,PANDA EXPRESS #2005,52.87,Food & Drink,Chase5282_Activity20240110_20260110_20260111.CSV,This is a list of notable current and former f...
246,2a01f41e-8f1e-4a10-9e7f-3ac29795607b,2025-06-14 00:00:00,Payment Thank You-Mobile,-62.07,None,Chase5282_Activity20240110_20260110_20260111.CSV,Locate a mobile phone | GSM locator of mobile ...
247,f7648015-85a6-4412-87d5-1499ac29fce6,2025-06-12 00:00:00,STARS AND STRIKES - HUNTS,21.80,Entertainment,Chase5282_Activity20240110_20260110_20260111.CSV,"At our Huntsville , AL location , we pride our..."
248,2edd49e5-9f4f-46d4-b94f-f9d7661bcdab,2025-06-11 00:00:00,WAL-MART #332,4.47,Groceries,Chase5282_Activity20240110_20260110_20260111.CSV,From toys and video games to fashionable cloth...


In [6]:
# df_view["amount"].sum()

In [7]:
db = SQLDatabase.from_uri("sqlite:///money_rag.db")

embeddings = VertexAIEmbeddings(model_name="text-embedding-005")


# Initialize Qdrant with disk persistence
# "path" creates a local directory to store the vectors
qdrant_client = QdrantClient(path="qdrant_db") 
COLLECTION_NAME = "transactions"

def sync_to_qdrant(db_path: str):
    """Sync the transactions table from SQLite to Qdrant vector store."""
    # Load data from the database
    conn = sqlite3.connect(db_path)
    # Ensure we select the new column
    df = pd.read_sql_query("SELECT * FROM transactions", conn)
    conn.close()

    # Recreate collection to ensure clean state on re-ingestion
    if qdrant_client.collection_exists(COLLECTION_NAME):
        qdrant_client.delete_collection(COLLECTION_NAME)
        
    qdrant_client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=768, distance=Distance.COSINE),
    )
    
    vector_store = QdrantVectorStore(
        client=qdrant_client,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
    )

    # Use description + enrichment as the main text for embedding
    texts = []
    for _, row in df.iterrows():
        # Combine original description with the permanent enriched info
        # If enriched_info is present, it looks like: "MCDONALDS - Fast food chain..."
        enriched = row.get('enriched_info', '')
        if enriched:
            texts.append(f"{row['description']} - {enriched}")
        else:
            texts.append(str(row['description']))
    
    # Store other fields as metadata for correlation
    metadatas = df[['id', 'amount', 'category', 'transaction_date']].to_dict('records')
    # Convert timestamps to string for metadata compatibility
    for m in metadatas:
        m['transaction_date'] = str(m['transaction_date'])

    vector_store.add_texts(texts=texts, metadatas=metadatas)
    print(f"✅ Synced {len(texts)} records to Qdrant at 'qdrant_db/'.")
    return vector_store

# Initialize store from DB
vector_store = sync_to_qdrant("money_rag.db")

/Users/sawale/Documents/learning/money_rag/.venv/lib/python3.12/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


✅ Synced 250 records to Qdrant at 'qdrant_db/'.


In [8]:
@dataclass
class RuntimeContext:
    db: SQLDatabase
    vector_store: QdrantVectorStore

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"


@tool
def semantic_search(query: str, topk: int = 5) -> str:
    """
    Search for transactions semantically when exact category or description matches are unknown.
    Use this to find specific merchants or types of spending (e.g., 'streaming services' or 'fast food').
    Returns a list of matching transactions with their IDs and metadata.
    """
    runtime = get_runtime(RuntimeContext)
    vs = runtime.context.vector_store
    results = vs.similarity_search(query, k=topk)
    
    output = []
    for doc in results:
        output.append(f"Result: {doc.page_content} | Metadata: {doc.metadata}")
    
    return "\n".join(output) if output else "No semantically similar transactions found."


SYSTEM = f"""You are a sophisticated financial analyst with access to both a SQLite database and a semantic search tool.

Workflow:
1. **Identify the Need**: If the user's request uses vague terms (e.g., "junk food", "travel stuff") or you don't know the exact category/description name, start with `semantic_search`.
2. **Semantic Discovery**:
    - Call `semantic_search(query, topk=...)`. 
    - **Pro Tip**: Use a higher `topk` (e.g., 10 or 20) if you suspect there are many relevant transactions to find.
    - **Relevance Check**: Not all results in the `topk` list may be relevant. You have permission to select only the few that match the user's intent and discard the rest.
    - **Reiteration Logic**: 
        - If the initial results seem too narrow, or if you suspect more relevant transactions exist but were cut off, reiterate by calling `semantic_search` again with a higher `topk` (e.g., 20, 50).
        - **Stop Condition**: If you found a few matching transactions and are confident that's all of them (i.e., the rest of the results are clearly irrelevant), do NOT iterate further.
3. **Filter & Extract**:
    - Review the results from semantic search. Manually filter out any that aren't relevant to the user's specific intent.
    - Extract the unique `id` values from the metadata of relevant results.
4. **Handling No Results**:
    - If `semantic_search` returns nothing useful, or if the results are clearly not what the user asked for (e.g., user asked for "coffee" but results are all "gas stations"), STOP.
    - Return a clear message: "I couldn't find any transactions related to [topic]."
    - **DO NOT GUESS**. It is better to say you found nothing than to sum up unrelated transactions.
5. **SQL Execution**:
    - Use `execute_sql` to perform the final calculation or retrieval.
    - Correlate results by using the discovered IDs in your query: `SELECT SUM(amount) FROM transactions WHERE id IN ('uuid1', 'uuid2', ...)`.
    - You can also use discovered merchant names if they share a common pattern.

Rules:
- **Thinking**: Explain your plan before calling any tools.
- **Read-only**: No modifications (INSERT/UPDATE/DELETE/etc.) to the database.
- **Spending Logic**: 
    - Spending = POSITIVE values (> 0). 
    - Payments/Refunds = NEGATIVE values (< 0). EXCLUDE negative values when calculating spending.
- **SQL Formatting**: Limit results to 5 rows for non-aggregation queries. Use `SUM()` for totals.
"""

agent = create_agent(
 model=vertex_llm,
 tools=[execute_sql, semantic_search],
 system_prompt=SYSTEM,
 context_schema=RuntimeContext,
 checkpointer=InMemorySaver(),
)

In [9]:
# # Initialize the store once
# # vector_store = sync_to_qdrant("money_rag.db")

# Test search
query = "Where did I spend money on groceries or food?"
results = vector_store.similarity_search(query, k=3)

for doc in results:
    print(f"Match: {doc.page_content} | Metadata: {doc.metadata}")

Match: CASHBACK BONUS REDEMPTION PYMT/STMT CRDT - What is this charge? Cashback bonus redemption pymt / stmt crdt . Cash - back from mogl 858-36958. His IP address, location , Internet provider, what browser he uses and operating system? You can find out all this using "2IP spy".It serves two main functions. It provides the network location of the host and identifies the host or network interface. What is the purpose of an IP address? Cashback bonus redemption pymt / stmt crdt .March is the month in which I had most number of transcations. I think this is because I visited the store next to me very frequently for small purchases. Cashback bonus redemption pymt / stmt crdt . Redemption Activity This Period is the total amount of Rewards you redeemed during the statement period and includes Miles partners, gift cards, account credits, electronic deposits and charitable donations. Cashback bonus redemption pymt / stmt crdt .Question2 : What was the frequency(number of occurences) of each 

In [10]:
question = "is there anything i need to be worried about in my spending over the last 2 months?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
    context=RuntimeContext(db=db, vector_store=vector_store)
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

is there anything i need to be worried about in my spending over the last 2 months?
================================== Ai Message ==================================

I can't tell you if there's anything to "worry about" as that's a subjective judgment. However, I can help you analyze your spending in various ways. For example, I can:

*   Show you your total spending over the last two months.
*   Break down your spending by category or merchant.
*   Identify your largest transactions.
*   Find transactions related to specific keywords (e.g., "fast food," "online shopping").

What kind of spending insights would be most helpful for you?


In [ ]:
# ...existing code...
# Interactive Chat Loop
print("💬 Chat with your financial data! (Type 'exit' to stop)")

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["exit", "quit", "q"]:
            print("Goodbye!")
            break
            
        print("\n" + "-"*50)
        
        # Stream the agent's response
        for step in agent.stream(
            {"messages": [{"role": "user", "content": user_input}]},
            {"configurable": {"thread_id": "1"}}, # Keeps memory of the conversation
            stream_mode="values",
            context=RuntimeContext(db=db, vector_store=vector_store)
        ):
            step["messages"][-1].pretty_print()
            
        print("-" * 50 + "\n")
        
    except KeyboardInterrupt:
        print("\nGoodbye!")
        break

💬 Chat with your financial data! (Type 'exit' to stop)

--------------------------------------------------
================================ Human Message =================================

or db won't be in github..
================================== Ai Message ==================================

It seems like your last message might be incomplete or a comment about something else. Could you please let me know what you'd like me to do or if you have a question about your spending? I'm ready to help analyze your transactions.
--------------------------------------------------

